In [ ]:
import sympy
from control.matlab import *
from IPython.display import Latex, Math, display
from sympy import Poly
from sympy.abc import s, z

In [8]:
%pylab %matplotlib inlinedef tex(eqn):    display(Math(sympy.latex(eqn)))

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [9]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

<div id="toc"></div>

# DC Motor Speed: PID Controller Design

From the main problem, the dynamic equations in the Laplace domain and the open-loop transfer function of the DC Motor are the following.

$$
s(Js + b)\Theta(s) = KI(s)
$$



$$
(Ls + R)I(s) = V(s) - Ks\Theta(s)
$$



$$
P(s) = \frac{\dot{\Theta}(s)}{V(s)} = \frac{K}{(Js + b)(Ls + R) + K^2}  \qquad [\frac{rad/sec}{V}]
$$

The structure of the control system has the form shown in the figure below.
![feedback_motors.png](figures/feedback_motors.png)
For the original problem setup and the derivation of the above equations, please refer to the [DC Motor Speed: System Modeling](../MotorSpeed/MotorSpeed-SystemModeling.ipynb) page. For a 1-rad/sec step reference, the design criteria are the following:

* Settling time less than 2 seconds
* Overshoot less than 5
* Steady-state error less than 1

Now let's design a controller using the methods introduced in the [Introduction: PID Controller Design](../Introduction/Introduction-ControlPID.ipynb) page.

In [10]:
J = 0.01
b = 0.1
K = 0.01
R = 1
L = 0.5
P_motor_sym = K / ((J * s + b) * (L * s + R) + K ** 2)
tex(P_motor_sym)

<IPython.core.display.Math object>

Recall that the transfer function for a PID controller is:

$$
C(s) = K_{p} + \frac {K_{i}} {s} + K_{d}s = \frac{K_{d}s^2 + K_{p}s + K_{i}} {s}
$$



## Proportional control

Let's first try employing a proportional controller with a gain of 100, that is, _C_(_s_) = 100. To determine the closed-loop transfer function, we use the |feedback| command.  Add the following code to the end of your m-file.

In [3]:
Kp = 100
C = Kp
# sys_cl = feedback(C*P_motor,1);

Now let's examine the closed-loop step response. Add the following commands to the end of your m-file and run it in the command window. You should generate the plot shown below. You can view some of the system's characteristics by right-clicking on the figure and choosing **Characteristics** from the resulting menu. In the figure below, annotations have specifically been added for **Settling Time**, **Peak Response**, and **Steady State**. <html> </p><pre class="codeinput"> t = 0:0.01:5; step(sys_cl,t) grid title('Step Response with Proportional Control') </pre> </html>
![sPID_control_01.png](figures/sPID_control_01.png)
From the plot above we see that both the steady-state error and the overshoot are too large. Recall from the [Introduction: PID Controller Design](../Introduction/Introduction-ControlPID.ipynb) page that increasing the proportional gain _Kp_ will reduce the steady-state error. However, also recall that increasing _Kp_ often results in increased overshoot, therefore, it appears that not all of the design requirements can be met with a simple proportional controller. This fact can be verified by experimenting with different values of _Kp_. Specifically, you can employ the **SISO Design Tool** by entering the command |sisotool(P_motor)| then opening a closed-loop step response plot from the **Analysis Plots** tab of the **Control and Estimation Tools Manager** window. With the **Real-Time Update** box checked, you can then vary the control gain in the **Compensator Editor** tab and see the resulting effect on the closed-loop step response. A little experimentation verifies what we anticipated, a proportional controller is insufficient for meeting the given design requirements; derivative and/or integral terms must be added to the controller.

## PID control

Recall from the [Introduction: PID Controller Design](../Introduction/Introduction-ControlPID.ipynb) page adding an integral term will eliminate the steady-state error to a step reference and a derivative term will often reduce the overshoot. Let's try a PID controller with small _Ki_ and _Kd_. Modify your m-file so that the lines defining your control are as follows. Running this new m-file gives you the plot shown below.

In [4]:
# Kp = 75;
# Ki = 1;
# Kd = 1;
# C = pid(Kp,Ki,Kd);
# sys_cl = feedback(C*P_motor,1);
# step(sys_cl,[0:1:200])
# title('PID Control with Small Ki and Small Kd')